# Data Modelling
This is a notebook to experiment with the data modelling of the sales quantity data.
This was done on a cloud instance so the file paths will be different if you are running this locally.
Note that the dataset is also propiertary so it will not be included in this repository.

# 1.Imports and Constants
We will be using the following libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
WINDOW = 20
BATCH_SIZE = 2048
BUFFER = 100000

# Load data

The data is a csv extracted from an SQL database and cleaned. It contains the following columns:
- **date:** The date of the sale
- **item_code:** The code of the product sold
- **quantity:** The quantity of the product sold on that day

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# %pwd

In [ ]:
# filepath = 'sales_quantity.csv' #for local imports
# filepath = '/home/jupyter/sales_quantity.csv' #vm-instance
filepath = '/content/drive/MyDrive/Documents/uni_work/Bangkit2023/batch1/capstone/repo/ml_modeling/sales_quantity.csv' #colab
data = pd.read_csv(filepath,names=['date','item_code','quantity'],
                   dtype = {'item_code':str, 'quantity':np.float64},header = 0 )
data.head()

# Transform data
We need to change the data so that it has the following format for training:
- **Input:** [*The tokenized item code, day, month, day of the week, day of the year, {A sequence of 20 days of sales data for a particular product}*]
- **Output:** The quantity sold for that product in the following day
>**Note:**
    - *The tokenized item code is the index of the item code in the tokenizer's word index.*
    - *The day component of the date is the day of the month.*
    - *The month component of the date is the month of the year.*
    - *The day of the week is a number between 0 and 6, where 0 is Monday and 6 is Sunday.*
    - *The day of the year is a number between 1 and 365, where January 1st is 1 and December 31st is 365.*
    - *The sequence of 20 days of sales data is the window size we will use for training the model. The data will be normalized*


In [ ]:
#extract date features from date column
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_year'] = data['date'].dt.dayofyear



We need to create a wide dataframe with each item code as a column and the quantity sold for each day as the values.

In [ ]:
#stack dataframe based on item_code
item_sales = data.groupby(['item_code','date','year','month','day','day_of_week','day_of_year'])['quantity'].sum().unstack(level=0)
#turn each NaN value to 0
item_sales = item_sales.sort_values('date')
item_sales = item_sales.fillna(0)
item_sales.reset_index(inplace=True)
item_sales

## Prepare item code and dates
Since the dataset will use date and item code feature as input, to create an array of item code mapped to every date value

In [ ]:
#prepare the list of item codes

items = np.array(item_sales.columns[6:])
total_items = items.shape[0]
print(items.shape)

In [ ]:
# prepare the array of date_related features, since we will be windowing these features
# we ignore the first few ones

dates = np.array(item_sales[['month','day','day_of_week','day_of_year']][WINDOW:])

#normalize for cyclic feature

dates = np.sin(dates) + np.cos(dates)
total_dates = dates.shape[0]
print(dates.shape)

Create numpy arrays for each repeated item and dates for later joining.

In [ ]:
repeated_items = items.repeat(total_dates)
repeated_dates = dates.reshape(1,dates.shape[0],dates.shape[1]).repeat(total_items,axis=0).reshape(-1,4)

print(repeated_items.shape)
print(repeated_dates.shape)

## Prepare the sales data to be windowed
We need to create windows of the sales data corresponding to the dates. This will be used as input and output for the data later on.

In [ ]:
#transpose the sales quantity so dates are columns
sales_quantity = np.array(item_sales.iloc[:,6:]).T
#create the windows
windowed = np.lib.stride_tricks.sliding_window_view(sales_quantity, WINDOW+1, axis=-1).reshape(-1,WINDOW+1)
print(f'Shape of windowed data {windowed.shape}')

#seperate the input and target
sales_input = windowed[:,:-1]
target = windowed[:,-1]
print(f'Shape of windowed input {sales_input.shape}')
print(f'Shape of target {target.shape}')

## Normalization
We need to normalize the input and output of the sales data. 
We can normalize the output in place while using a normalization layer to normalize the input



# Model Analysis

Since we will be passing the item code as a feature to the model, we need to tokenize it.
We use a helper function to create a tokenizer and fit it on the item codes.
This will be also be used to later decode the predictions of the model.

In [ ]:

def create_vectorize(item_code):
    """
    Create a tokenizer to tokenize item codes.

    Args:
        item_code (list or Series): List or Series containing item codes.

    Returns:
        tf.keras.preprocessing.text.Tokenizer: Tokenizer object fitted on item codes.
    """

    # Create a tokenizer with no filters and case-sensitive tokenization
    vectorize_layer = tf.keras.layers.StringLookup()

    # Fit the tokenizer on the item codes
    vectorize_layer.adapt(item_code)

    return vectorize_layer

# Create a tokenizer using item codes from item_sales dataframe columns
vectorize_layer = create_vectorize(tf.constant(item_sales.columns[6:]))

# Get the length of the tokenizer's word index
tokenizer_word_count = vectorize_layer.vocabulary_size()
print(f'Tokenizer has {tokenizer_word_count} tokens')

After preparing the dataset, we try to fine tune the learning rate of the algorithm.
We only use a sample of the training set to speed up the process.

In [ ]:
#create a list of models based on given learning rates and optimizers
def create_model():
    """
    Creates a list of models based on the learning rates and optimizers given.

    Args:
        learning_rate_array (list): A list of learning rates and optimizers to use for each model.

    Returns:
        list: A list of models.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                               strides=1,
                               activation="relu", padding="causal",
                               input_shape=[25, 1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    return model


In [ ]:
model = create_model()
train_sample = train_set.unbatch().shuffle(BUFFER).take(train_len//10).batch(BATCH_SIZE).prefetch(1)
val_sample = val_set.unbatch().shuffle(BUFFER).take(val_len//10).batch(BATCH_SIZE).prefetch(1)
train_sample

In [ ]:
#callback to tune the learning rate
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

optimizer = tf.keras.optimizers.SGD(momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mape"])

history = model.fit(train_sample, epochs=100, callbacks=[lr_schedule],validation_data = val_sample, verbose=2)


In [ ]:
# Define the learning rate array
lrs = 1e-8 * (10 ** (np.arange(100) / 20))

# Set the figure size
plt.figure(figsize=(10, 6))

# Set the grid
plt.grid(True)

# Plot the loss in log scale
plt.semilogx(lrs, history.history["loss"])

# Increase the tickmarks size
plt.tick_params('both', length=10, width=1, which='both')
plt.xlabel("Learning rate")
plt.ylabel("Loss")


# Model Training

We found that the best learning rate is 10e-5 (or 1e-4).

In [ ]:
#Reset the states generated by keras
tf.keras.backend.clear_session()

model = create_model()

In [ ]:
#set the learning rate
learning_rate = 1e-4
#set the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

#set the callback to stop the training if the validation loss doesn't improve
callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mape"])

history = model.fit(train_set, epochs=500, validation_data = val_set, verbose=2, callbacks=[callback])

In [ ]:
evaluation = model.evaluate(test_set)